In [1]:
#载入数据处理必备的模组
import numpy as np
import pandas as pd

In [3]:
#载入数据
data=pd.read_csv("data/data.csv")
print(data.head())
print(len(data))



  stock_code       revenue  s_fa_eps_basic     du_profit  plus_net_invest_inc  \
0  000001.SZ  8.196800e+10          1.0600  0.000000e+00         0.000000e+00   
1  000002.SZ  1.170000e+11          1.0050  2.222772e+10         2.046834e+09   
2  000008.SZ  1.007329e+09          0.0434  1.776341e+08         0.000000e+00   
3  000060.SZ  1.482919e+10          0.3800  1.279332e+09         2.853988e+07   
4  000063.SZ  4.775730e+10          0.9300  5.884933e+09        -1.951700e+07   

   net_cash_flows_oper_act  net_cash_flows_inv_act  net_roe  total_roe  \
0            -1.530000e+11           -9.716200e+10   8.7813     0.6289   
1             6.474047e+09            1.392036e+10   9.5305     1.6026   
2            -9.947078e+07           -1.193312e+09   1.9528     1.5235   
3             7.571307e+08           -4.932751e+08   8.6684     4.9238   
4            -3.170483e+09           -4.387188e+09  12.6504     2.9854   

         RSI       VOL      WILLR         y      date  
0  52.970207

In [6]:
#分析数据的日期
date=data.loc[:,"date"]
#print(type(date))
date=date.tolist()
#print(type(date))
date=set(date)
date=list(date)
date=sorted(date)
print(date)

<class 'pandas.core.series.Series'>
<class 'list'>
[20171207, 20180105, 20180209, 20180309, 20180412, 20180510, 20180608, 20180706, 20180803, 20180831, 20181006, 20181103, 20181201, 20190102, 20190206, 20190306, 20190404, 20190507, 20190605, 20190703, 20190731, 20190829, 20191003, 20191031, 20191128, 20191227, 20200201]


In [7]:
#准备数据输入和标签
data=data.set_index(["stock_code"])
#数据清洗，去除NAN
data=data.dropna()
x_data=data.drop(["date","y"],axis=1)  # axis=1代表将列删掉；0代表将行删掉
y_data=data.loc[:,"y"]

In [12]:
#对于数据进行预处理
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

#对输入数据进行处理，提取可以被机器学习接受的数组形式
#print(type(x_data))
#print(x_data.head())
x=x_data.values
#print(type(x))
#print(x)
#对数据进行归一化处理
nm_scaler=MinMaxScaler()
x=nm_scaler.fit_transform(x)

# 使用主成分分析，对数据作降维处理，提取10个关键的特征

pca=PCA(n_components=10)
x=pca.fit_transform(x)



#划分训练和测试数据集

train_x=x[0:5702]
test_x=x[5702:]



<class 'pandas.core.frame.DataFrame'>
                 revenue  s_fa_eps_basic     du_profit  plus_net_invest_inc  \
stock_code                                                                    
000001.SZ   8.196800e+10          1.0600  0.000000e+00         0.000000e+00   
000002.SZ   1.170000e+11          1.0050  2.222772e+10         2.046834e+09   
000008.SZ   1.007329e+09          0.0434  1.776341e+08         0.000000e+00   
000060.SZ   1.482919e+10          0.3800  1.279332e+09         2.853988e+07   
000063.SZ   4.775730e+10          0.9300  5.884933e+09        -1.951700e+07   

            net_cash_flows_oper_act  net_cash_flows_inv_act  net_roe  \
stock_code                                                             
000001.SZ             -1.530000e+11           -9.716200e+10   8.7813   
000002.SZ              6.474047e+09            1.392036e+10   9.5305   
000008.SZ             -9.947078e+07           -1.193312e+09   1.9528   
000060.SZ              7.571307e+08           -4

In [14]:
#对标签数据进行处理
y=y_data.copy()
y=y.values

#把结果二值化用于分类的标签

y[y>=0.04]=1
y[y<0.04]=0
#print(y)

#划分训练和测试标签集

train_y=y[0:5702]
test_y=y[5702:]





[0. 0. 1. ... 1. 1. 1.]


In [15]:
# 构建最基础的机器学习KNN模型，对模型进行训练
from sklearn.neighbors import KNeighborsClassifier          #knn是最基础的机器学习模型，他的预测取决于当前的情况和历史上的情况进行对比，找到最相似的进行推断
knn=KNeighborsClassifier(n_neighbors=3,weights="distance")  #跟历史上最相似的3次情况进行对比，求欧式距离找到距离最近的情况

knn.fit(train_x,train_y)
predict_train_y=knn.predict(train_x)




In [16]:
print(predict_train_y)

[0. 0. 1. ... 0. 0. 0.]


In [17]:
# 查看训练的拟合度

from sklearn.metrics import accuracy_score, confusion_matrix

print(accuracy_score(train_y,predict_train_y))
cm=confusion_matrix(train_y,predict_train_y)
print(cm)


1.0
[[4263    0]
 [   0 1439]]


In [18]:
# 查看测试结果
predict_test_y=knn.predict(test_x)

print(accuracy_score(test_y,predict_test_y))
cm=confusion_matrix(test_y,predict_test_y)
print(cm)





0.5925414364640884
[[1130  228]
 [ 657  157]]


In [20]:
#搭建ANN模型，载入必须的库  人工神经网络（Artificial Neural Network，简称ANN ）

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import adam_v2


In [27]:
#搭建模型

model=Sequential([Dense(10,input_dim=10),Activation("tanh"),Dense(1),Activation("sigmoid")])

adam=adam_v2.Adam(lr=0.01)
model.compile(optimizer=adam,loss="mean_squared_error",metrics=["accuracy"])



In [28]:
#训练模型
model.fit(train_x,train_y,epochs =100,batch_size=500)


Epoch 1/100
12/12 [==============================] - 0s 1ms/step - loss: 0.2256 - accuracy: 0.6991
Epoch 2/100
12/12 [==============================] - 0s 909us/step - loss: 0.1894 - accuracy: 0.7476
Epoch 3/100
12/12 [==============================] - 0s 653us/step - loss: 0.1858 - accuracy: 0.7476
Epoch 4/100
12/12 [==============================] - 0s 545us/step - loss: 0.1847 - accuracy: 0.7476
Epoch 5/100
12/12 [==============================] - 0s 455us/step - loss: 0.1836 - accuracy: 0.7480
Epoch 6/100
12/12 [==============================] - 0s 727us/step - loss: 0.1828 - accuracy: 0.7478
Epoch 7/100
12/12 [==============================] - 0s 636us/step - loss: 0.1824 - accuracy: 0.7482
Epoch 8/100
12/12 [==============================] - 0s 455us/step - loss: 0.1820 - accuracy: 0.7494
Epoch 9/100
12/12 [==============================] - 0s 545us/step - loss: 0.1816 - accuracy: 0.7515
Epoch 10/100
12/12 [==============================] - 0s 364us/step - loss: 0.1814 - accuracy

12/12 [==============================] - 0s 455us/step - loss: 0.1801 - accuracy: 0.7508
Epoch 82/100
12/12 [==============================] - 0s 636us/step - loss: 0.1801 - accuracy: 0.7506
Epoch 83/100
12/12 [==============================] - 0s 545us/step - loss: 0.1801 - accuracy: 0.7499
Epoch 84/100
12/12 [==============================] - 0s 545us/step - loss: 0.1800 - accuracy: 0.7510
Epoch 85/100
12/12 [==============================] - 0s 545us/step - loss: 0.1801 - accuracy: 0.7511
Epoch 86/100
12/12 [==============================] - 0s 455us/step - loss: 0.1800 - accuracy: 0.7494
Epoch 87/100
12/12 [==============================] - 0s 546us/step - loss: 0.1803 - accuracy: 0.7506
Epoch 88/100
12/12 [==============================] - 0s 455us/step - loss: 0.1800 - accuracy: 0.7499
Epoch 89/100
12/12 [==============================] - 0s 545us/step - loss: 0.1799 - accuracy: 0.7499
Epoch 90/100
12/12 [==============================] - 0s 455us/step - loss: 0.1800 - accuracy: 

In [29]:
predict_test_y=model.predict(test_x)
print(predict_test_y)

[[0.19518653]
 [0.12833136]
 [0.19710451]
 ...
 [0.3385986 ]
 [0.25403118]
 [0.23298374]]


In [30]:
#测试模型表现
from sklearn.metrics import accuracy_score, confusion_matrix
#设定阈值
predict_test_y=model.predict(test_x)
theta=0.55
predict_test_y[predict_test_y>=theta]=1
predict_test_y[predict_test_y<theta]=0


#将量化预测结果进行阈值分类

acc=accuracy_score(test_y,predict_test_y)
cm=confusion_matrix(test_y,predict_test_y)

print(acc)
print(cm)


#查看统计结果


0.6284530386740331
[[1357    1]
 [ 806    8]]
